In [20]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

transactions = pd.read_csv('../../res/credit_card_fraud.csv', sep=';')


,transactionId,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0,1,PAYMENT,"9839,64",C1231006815,170136,"160296,36",M1979787155,0,0,0
1,1,1,PAYMENT,"1864,28",C1666544295,21249,"19384,72",M2044282225,0,0,0
2,2,1,TRANSFER,181,C1305486145,181,0,C553264065,0,0,1
3,3,1,CASH_OUT,181,C840083671,181,0,C38997010,21182,0,1
4,4,1,PAYMENT,"11668,14",C2048537720,41554,"29885,86",M1230701703,0,0,0
